In [ ]:
!pip install transformers==3.5.0
!pip install nltk
!pip install biobert-pytorch
!pip install wget
!pip install cookiecutter

     |████████████████████████████████| 1.3MB 11.6MB/s 
     |████████████████████████████████| 2.9MB 45.8MB/s 
     |████████████████████████████████| 1.1MB 56.9MB/s 
     |████████████████████████████████| 890kB 53.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ee5ae7e0d502d56f398a45e5a35ce81230f3ed37a71fa403e6428512467abf49
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=38d5b6bcd2b8833b338306ece5dd0175e00701233dd0aeb0f4cb4fbcb803dfb4
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
     |████████████████████████████████| 51kB 8.8MB/s 


In [ ]:
import os
import io
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertForPreTraining
import nltk
from sklearn.model_selection import train_test_split
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/MeasEval-main/data/train/train/text

/content/drive/My Drive/MeasEval-main/data/train/train/text


In [ ]:
files=os.listdir()

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):

    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """
    # print(len(sentence))
    # print(len(text_labels))
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)
    # print(len(tokenized_sentence))
    # print(len(labels))
    return tokenized_sentence, labels

In [ ]:
cd /content/drive/My Drive/MeasEval-main/data/train/train/

/content/drive/My Drive/MeasEval-main/data/train/train


In [ ]:
l=0
NERdata=[]
for i in files:

  pathtxt=os.path.join('text',i)
  
  with open(pathtxt, 'r') as f:
    text=f.read()
  pathtsv=os.path.join('tsv',i[:-3]+'tsv')
  if(not(os.path.exists(pathtsv))):
    continue
  filetsv=pd.read_csv(pathtsv,sep = '\t')
  entity=filetsv[['annotType','startOffset','endOffset','text']]
  entity = entity.to_numpy().tolist()

  ent=0
  offset=0
  sent_text = nltk.sent_tokenize(text)
  
  for j in sent_text:
    row=[]
    row.append(i[:-4])
    
    
    ent_sen=[]
    ret = [(m.group(0), m.start(), m.end() - 1) for m in re.finditer(r'\S+', j)]
    start=[]
    
    for word in ret:
      types=[]
      for ent_doc in entity:
        if(offset+word[1]>=ent_doc[1] and offset+word[2]<=ent_doc[2]):
          types.append(ent_doc[0])

      ent_sen.append(types)     
    
    
    sen,label=tokenize_and_preserve_labels(j.split(' '),ent_sen,tokenizer)
    label.insert(0,[])
    label.append([])
    offset+=(1+len(j))

    row.append(j)
    row.append(label)
    NERdata.append(row)

In [ ]:
cd /content/drive/My Drive/MeasEval-main/data/train/dev/text

/content/drive/My Drive/MeasEval-main/data/train/dev/text


In [ ]:
files_val=os.listdir()

In [ ]:
cd /content/drive/My Drive/MeasEval-main/data/train/dev

/content/drive/My Drive/MeasEval-main/data/train/dev


In [ ]:
l=0
NERdata_val=[]

for i in files_val:
  
  pathtxt=os.path.join('text',i)
  
  with open(pathtxt, 'r') as f:
    text=f.read()
  pathtsv=os.path.join('tsv',i[:-3]+'tsv')
  if(not(os.path.exists(pathtsv))):
    continue
  filetsv=pd.read_csv(pathtsv,sep = '\t')
  entity=filetsv[['annotType','startOffset','endOffset','text']]
  entity = entity.to_numpy().tolist()

  ent=0
  offset=0
  sent_text = nltk.sent_tokenize(text)
  
  for j in sent_text:
    row=[]
    row.append(i[:-4])
    
    
    ent_sen=[]
    ret = [(m.group(0), m.start(), m.end() - 1) for m in re.finditer(r'\S+', j)]
    start=[]
    
    for word in ret:
      types=[]
      for ent_doc in entity:
        if(offset+word[1]>=ent_doc[1] and offset+word[2]<=ent_doc[2]):
          types.append(ent_doc[0])

      ent_sen.append(types)     
    
    
    sen,label=tokenize_and_preserve_labels(j.split(' '),ent_sen,tokenizer)
    label.insert(0,[])
    label.append([])
    offset+=(1+len(j))

    row.append(j)
    row.append(label)
    NERdata_val.append(row)

In [ ]:
print(len(NERdata))
print(len(NERdata_val))

1370
199


In [ ]:
NERdf=pd.DataFrame (NERdata, columns = ['Id','Text','Entity'])
NERdf_val=pd.DataFrame (NERdata_val, columns = ['Id','Text','Entity'])

In [ ]:
NERdf.head()

,Id,Text,Entity
0,S0006322312001096-1221,"We constructed Framingham CVD, CHD, and stroke...","[[], [], [], [], [], [], [], [], [], [], [], [..."
1,S0006322312001096-1221,"Crude, age- and sex-adjusted, and multivariabl...","[[], [], [], [], [], [], [], [], [], [], [], [..."
2,S0006322312001096-1221,"In the analysis of GHQ symptoms, the sample wa...","[[], [], [], [], [], [], [], [], [], [], [], [..."
3,S0006322312001096-1221,For depressive symptoms defined by CES-D and a...,"[[], [], [MeasuredEntity], [MeasuredEntity], [..."
4,S0167610513002729-1127,The situation for the 45° case is more interes...,"[[], [], [], [], [], [Quantity], [Quantity], [..."


In [ ]:
def len_text(x):
  sum = 0
  for en in x:
    sum += len(en)
  return sum

In [ ]:
NERdf["len"] = NERdf["Entity"].apply(lambda x: len_text(x))

In [ ]:
NERdf.head()

,Id,Text,Entity,len
0,S0006322312001096-1221,"We constructed Framingham CVD, CHD, and stroke...","[[], [], [], [], [], [], [], [], [], [], [], [...",0
1,S0006322312001096-1221,"Crude, age- and sex-adjusted, and multivariabl...","[[], [], [], [], [], [], [], [], [], [], [], [...",10
2,S0006322312001096-1221,"In the analysis of GHQ symptoms, the sample wa...","[[], [], [], [], [], [], [], [], [], [], [], [...",35
3,S0006322312001096-1221,For depressive symptoms defined by CES-D and a...,"[[], [], [MeasuredEntity], [MeasuredEntity], [...",26
4,S0167610513002729-1127,The situation for the 45° case is more interes...,"[[], [], [], [], [], [Quantity], [Quantity], [...",3


In [ ]:
NERdf_z = NERdf[NERdf["len"] == 0].sample(500)

In [ ]:
NERdf_nz = NERdf[NERdf["len"] != 0]

In [ ]:
NERdf_nz.head()

,Id,Text,Entity,len
1,S0006322312001096-1221,"Crude, age- and sex-adjusted, and multivariabl...","[[], [], [], [], [], [], [], [], [], [], [], [...",10
2,S0006322312001096-1221,"In the analysis of GHQ symptoms, the sample wa...","[[], [], [], [], [], [], [], [], [], [], [], [...",35
3,S0006322312001096-1221,For depressive symptoms defined by CES-D and a...,"[[], [], [MeasuredEntity], [MeasuredEntity], [...",26
4,S0167610513002729-1127,The situation for the 45° case is more interes...,"[[], [], [], [], [], [Quantity], [Quantity], [...",3
7,S0167610513002729-1127,"In this snapshot, at z=0.23 H the whole plume ...","[[], [], [], [Qualifier], [Qualifier], [Qualif...",4


In [ ]:
NERdf = pd.concat([NERdf_nz, NERdf_z])

In [ ]:
len(NERdf)

1046

In [ ]:
text_val=NERdf_val['Text'].tolist()
textlabels_val=NERdf_val['Entity'].tolist()

for i in range(len(textlabels_val)):
  textlabels_val[i]=(textlabels_val[i] + 512 * [[]])[:512]
  print(textlabels_val[i])

[[], [], [], [], [], [], [], [], [], [], [], [], [], ['Quantity'], ['MeasuredEntity'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [

In [ ]:
text=NERdf['Text'].tolist()
textlabels=NERdf['Entity'].tolist()

for i in range(len(textlabels)):
  textlabels[i]=(textlabels[i] + 512 * [[]])[:512]
  print(textlabels[i])

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['Quantity'], ['Quantity'], ['MeasuredEntity'], ['MeasuredEntity'], [], ['Quantity'], ['Quantity'], ['MeasuredProperty'], ['MeasuredProperty'], [], ['MeasuredEntity'], ['MeasuredEntity'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],

In [ ]:
inputs = tokenizer(text,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")

In [ ]:
inputs_val = tokenizer(text_val,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")

In [ ]:
import numpy as np

In [ ]:
train_seq = np.array(inputs['input_ids'])
train_mask = np.array(inputs['attention_mask'])
val_seq = np.array(inputs_val['input_ids'])
val_mask = np.array(inputs_val['attention_mask'])

In [ ]:
#labels = {"Quantity":0, "MeasuredEntity": 1, "MeasuredProperty": 2, "Qualifier": 3}
labels = {"Quantity": 0}

In [ ]:
onehot_labels = np.zeros((len(textlabels), len(textlabels[0]),1));
for i in range(len(textlabels)):
  for j in range(len(textlabels[0])):
      for k in range(len(textlabels[i][j])):
        if textlabels[i][j][k] =="Quantity":
          onehot_labels[i][j][labels[textlabels[i][j][k]]] = 1

In [ ]:
onehot_labels_val = np.zeros((len(textlabels_val), len(textlabels_val[0]),1));
for i in range(len(textlabels_val)):
  for j in range(len(textlabels_val[0])):
      for k in range(len(textlabels_val[i][j])):
        if textlabels_val[i][j][k] =="Quantity":
          onehot_labels_val[i][j][labels[textlabels_val[i][j][k]]] = 1

In [ ]:
train_y = onehot_labels
val_y = onehot_labels_val

In [ ]:
np.shape(val_y)

(199, 512, 1)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
train_data = TensorDataset(torch.from_numpy(train_seq), torch.from_numpy(train_mask),torch.from_numpy(train_y))
val_data = TensorDataset(torch.from_numpy(val_seq), torch.from_numpy(val_mask),torch.from_numpy(val_y))

In [ ]:
batch_size = 64
train_loader = DataLoader(train_data, shuffle=True, batch_size = batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size = batch_size)

In [ ]:
from transformers import BertTokenizer, BertModel
from transformers import BertForTokenClassification, AdamW
import torch
import torch.nn as nn
from transformers import BertForTokenClassification, AdamW

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert, embed_dim, hidden_dim, drop_prob, n_layers, out_dim):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      self.bilstm = nn.LSTM(embed_dim, hidden_dim,  bidirectional=True, batch_first=True)
      self.dropout = nn.Dropout(drop_prob)
      self.fc1 = nn.Linear(2*hidden_dim,out_dim)
      self.sigmoid = nn.Sigmoid()

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      x = self.bert(sent_id, attention_mask=mask)
      embed = x[0]
      cls_hs = x[1]
      x,_ = self.bilstm(embed)
      x = self.dropout(x)
      x = self.fc1(x)
      x = self.sigmoid(x)

      return x

In [ ]:
bert_model = BERT_Arch(model, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)

In [ ]:
def weighted_binary_cross_entropy(output, target, weights=None):
    output = torch.clamp(output,min=1e-8,max=1-1e-8)

    if weights is not None:
        assert len(weights) == 2
        loss = weights[1] * (target * torch.log(output)) + \
               weights[0] * ((1 - target) * torch.log(1 - output))
    else:
        loss = target * torch.log(output) + (1 - target) * torch.log(1 - output)

    return torch.neg(torch.mean(loss))

In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(bert_model.parameters(),lr = 0.005)

In [ ]:
bert_model = torch.load("/content/drive/My Drive/Model/quantity_mixedloss_t55_sh.pt")

In [ ]:
pos=0
neg=0
for seq, mask, y in val_loader:
    bert_model.zero_grad()
    bert_model.eval()
    y_pred = bert_model(seq.to(device), mask.to(device))
    np_out = y_pred.cpu().data.numpy()
    np_act = y.cpu().data.numpy()
    # for i in range(0, len(np_act[0])):
    #   print(np_act[0][i])
    #   print(np_out[0][i]>=0.3)
    #   print()
    # break
    #print(np.where(np_act==1))
    
    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          if np_out[i][j][k]>=0.55:
            np_out[i][j][k]=1
          else:
            np_out[i][j][k]=0
    
    
    #print(np.where(np_out==1))
    #print("##########")
    #print(np.where(np_act==1))

    # for i in range(np_out.shape[0]):
    #   for j in range(np_out.shape[1]):
    #     if np.max(np_out[i,j,:])==1 or np.max(np_act[i,j,:])==1:
    #       if np.max(np_out[i,j,:])==1 and np.max(np_act[i,j,:])==1:
    #         pos = pos + 1
    #       else:
    #         neg = neg + 1

    for i in range(np_out.shape[0]):
     for j in range(np_out.shape[1]):
       for k in range(np_out.shape[2]):
         if (np_out[i,j,k])==1 or (np_act[i,j,k])==1:
           if (np_out[i,j,k])==1 and (np_act[i,j,k])==1:
             pos = pos + 1
           else:
             neg = neg + 1

print(pos/(pos+neg))

0.8151041666666666


In [ ]:
p=0
n=0
pos=0
neg=0
prec_num = 0 
prec_den = 0
rec_num = 0
rec_den = 0 
p1 = 0
n1=0
for seq, mask, y in val_loader:
    bert_model.zero_grad()
    bert_model.eval()
    y_pred = bert_model(seq.to(device), mask.to(device))
    np_out = y_pred.cpu().data.numpy()
    np_act = y.cpu().data.numpy()
    #print(np_out)
    
    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          if np_out[i][j][k]>=0.55:
            np_out[i][j][k]=1
          else:
            np_out[i][j][k]=0

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        if np.max(np_out[i,j,:])==1 or np.max(np_act[i,j,:])==1:
          if np.max(np_out[i,j,:])==1 and np.max(np_act[i,j,:])==1:
            p1 = p1 + 1
          else:
            n1 = n1 + 1

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          if np_out[i,j,k] == np_act[i,j,k]:
            p = p +1
          else:
            n=n+1
          if (np_out[i,j,k])==1 or (np_act[i,j,k])==1:
            if (np_out[i,j,k])==1 and (np_act[i,j,k])==1:
              pos = pos + 1
            else:
              neg = neg + 1
          if np_out[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              prec_num = prec_num + 1
            else:
              prec_den = prec_den + 1
          if np_act[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              rec_num = rec_num + 1
            else:
              rec_den = rec_den + 1

precision = prec_num/(prec_num+prec_den)
recall = rec_num/(rec_num+rec_den)
F1 = 2*precision*recall/(precision+recall)

print("Entity recognition modified accuracy:-" + str(p1/(p1+n1)))
print("--------NER RESULTS--------")
print("Accuracy:-" + str(p/(n+p)))
print("Modified Accuracy:-" + str(pos/(pos+neg)))
print("Precision:-" + str(precision))
print("Recall:-" + str(recall))
print("F1 score:-"+str(F1))

Entity recognition modified accuracy:-0.8151041666666666
--------NER RESULTS--------
Accuracy:-0.9993031564070352
Modified Accuracy:-0.8151041666666666
Precision:-0.9072463768115943
Recall:-0.8892045454545454
F1 score:-0.8981348637015784
